In [ ]:
# A Script for calculating the dropout parameters through the factorized likelihood approach. 
# Plotting of resampled amplitude distribution, dropout parameters included

%matplotlib inlinez
%config InlineBackend.figure_format = 'retina'
#%load_ext line_profiler

from __future__ import division

import numpy as np
import glob
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import scipy.linalg as sl
import re

import enterprise
from enterprise.pulsar import Pulsar
from enterprise.pulsar import Tempo2Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
# from enterprise_extensions.models import model_general
from enterprise_extensions import sampler
from enterprise_extensions import model_utils
from enterprise_extensions import deterministic
from enterprise_extensions.timing import timing_block
from enterprise_extensions.blocks import (white_noise_block, red_noise_block,
                                          dm_noise_block,
                                          chromatic_noise_block,
                                          common_red_noise_block)
from enterprise_extensions.chromatic.solar_wind import solar_wind_block
from enterprise_extensions import chromatic as chrom
from enterprise_extensions import dropout as do
import enterprise_extensions
from enterprise.signals import anis_coefficients as anis

# from libstempo.libstempo import *
import cython
# import libstempo
import json
import acor
import scipy
import pickle
import copy
import json
import la_forge.core as co
import la_forge
from la_forge import slices
import healpy as hp


import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

Here skymap is an array of size (# of frequency bins, # of healpy pixels) and contains a map for all frequency bins, where each pixel is the square root of the sum of angle averaged h^2 values in that frequency bin and sky pixel.  

F_bins contains the edges of the frequency bins, and hist is the total number of binaries in each frequency bin for reference. I made this based on a 15 year timespan, which sets the lowest freqeuncy bin.


Let me know if you have any questions. I can also easily create these with a different nside if you think a different value would work better for what you want to do with these. And I can also make multiple realizations, which I assume we will want to get an averaged result, but I figured a single realization should be enough for first tests.

# Plotting Skymap from Holodeck simulated sky, other data tests

In [ ]:
npzfile = np.load("skymaps_for_steve_and_levi_nside32_1realization.npz")
skymaps = npzfile['skymaps']
F_bins = npzfile['F_bins']
hist = npzfile['hist']

In [ ]:
for k in npzfile.iterkeys():
    print(k)

In [ ]:
Tspan_15yr = 505861299.1401644

In [ ]:
print(((F_bins[1]-F_bins[0])**(-1))/3.15e7)
print(((F_bins[-1]-F_bins[-2])**(-1))/3.15e7)

In [ ]:
print(np.diff(F_bins)**(-1)/3.15e7)

In [ ]:
print(F_bins[:]**(-1)/3.15e7)

In [ ]:
print(Tspan_15yr/3.15e7)

In [ ]:
freqs = np.zeros((1,20))
for i in range(20):
    freqs[0,i] = i+1 / Tspan_15yr
    
print(freqs)

In [ ]:
print(skymaps.shape)

In [ ]:
map_test = np.sum(skymaps[:,:], axis=0)

In [ ]:
print(map_test.shape)

In [ ]:

hp.mollview(map_test, title="skymap")
hp.graticule()

# Getting Sph. Harm. Coefs for Skymap, Reconstruction

In [ ]:
coef_lmax8 = anis.clmFromMap(map_test, 1)

In [ ]:
print(coef_lmax8)

In [ ]:
map_from_coef_lmax8 = anis.mapFromClm(coef_lmax8, 8)

In [ ]:
print(len(map_from_coef_lmax8))

In [ ]:
hp.mollview(map_from_coef_lmax8, title="skymap")
hp.graticule()

In [ ]:
c_l_lmax8 = hp.anafast(map_test, lmax=8)
e_l_lmax8 = np.arange(len(c_l_lmax8))

# Angular Power Spectra over all Frequencies

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(e_l_lmax8, e_l_lmax8 * (e_l_lmax8 + 1) * c_l_lmax8)
plt.xlabel("$\ell$")
plt.ylabel("$\ell(\ell+1)C_{\ell}$")
plt.grid()
hp.write_cl("cl.fits", c_l_lmax8, overwrite=True)

In [ ]:
print(skymaps.shape)

# Per Frequency Skymaps

In [ ]:
map_20freqs = np.zeros((20, 12288))

for i in range(20): 
    map_20freqs[i] = skymaps[i,:]


In [ ]:


fig, ((ax0, ax1), (ax2, ax3), (ax4, ax5), (ax6, ax7), (ax8, 
      ax9), (ax10, ax11), (ax12, ax13), (ax14, ax15), (ax16, 
      ax17), (ax18, ax19)) = plt.subplots(nrows=10, ncols=2)

ax_list = [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, 
      ax9, ax10, ax11, ax12, ax13, ax14, ax15, ax16, 
      ax17, ax18, ax19]

for i, a in enumerate(ax_list):
    plt.axes(a)
    hp.mollview(map_20freqs[i], title="skymap_freqbin{}".format(i), hold=True)

fig.set_size_inches(8, 40)
# for a in range(2):
#     if a == 0:
#         for b in range(10):
#             plt.axes[b, a] = hp.mollview(map_20freqs[b], title="skymap_freqbin{}".format(b), hold=True)
#     if a == 1:
#         for c in range(10,19):
#             axs[c, a] = hp.mollview(map_20freqs[c], title="skymap_freqbin{}".format(c), hold=True)
#hp.mollview(map_testf0, title="skymap")
#hp.graticule()

In [ ]:
last5freqtest = anis.clmFromMap(map_20freqs[-5], lmax=8)

In [ ]:
print(last5freqtest)

In [ ]:
b=-4
hp.mollview(map_20freqs[b], title="skymap_freqbin{}".format(b))
hp.graticule()
fig.set_size_inches(6, 10)


In [ ]:
c_l_lmax8_20freqs = np.zeros((20, 9))
e_l_lmax8_20freqs = np.zeros((20, 9))


for i in range(20): 
    c_l_lmax8_20freqs[i] = hp.anafast(map_20freqs[i], lmax=8)
    e_l_lmax8_20freqs[i] = np.arange(len(c_l_lmax8_20freqs[i]))

In [ ]:
print(c_l_lmax8_20freqs[-6:])

In [ ]:
figps, ((axps0, axps1), (axps2, axps3), (axps4, axps5), (axps6, axps7), (axps8, 
      axps9), (axps10, axps11), (axps12, axps13), (axps14, axps15), (axps16, 
      axps17), (axps18, axps19)) = plt.subplots(nrows=10, ncols=2)

axps_list = [axps0, axps1, axps2, axps3, axps4, axps5, axps6, axps7, axps8, 
      axps9, axps10, axps11, axps12, axps13, axps14, axps15, axps16, 
      axps17, axps18, axps19]

for i, a in enumerate(axps_list):
    plt.axes(a)
    plt.plot(e_l_lmax8_20freqs[i], e_l_lmax8_20freqs[i] * 
             (e_l_lmax8_20freqs[i] + 1) * c_l_lmax8_20freqs[i])
    plt.xlabel("$\ell$")
    plt.ylabel("$\ell(\ell+1)C_{\ell}$")
    plt.grid()
figps.set_size_inches(8, 40)

In [ ]:
# figps, ((axps0, axps1), (axps2, axps3), (axps4, axps5), (axps6, axps7), (axps8, 
#       axps9), (axps10, axps11), (axps12, axps13), (axps14, axps15), (axps16, 
#       axps17), (axps18, axps19)) = plt.subplots(nrows=10, ncols=2)

# axps_list = [axps0, axps1, axps2, axps3, axps4, axps5, axps6, axps7, axps8, 
#       axps9, axps10, axps11, axps12, axps13, axps14, axps15, axps16, 
#       axps17, axps18, axps19]

figps1 = plt.figure()
axps01 = figps1.add_subplot(111)

for i in range(20):
    plt.plot(e_l_lmax8_20freqs[i], e_l_lmax8_20freqs[i] * 
             (e_l_lmax8_20freqs[i] + 1) * c_l_lmax8_20freqs[i], label='freq='+str(i))
    plt.xlabel("$\ell$")
    plt.ylabel("$\ell(\ell+1)C_{\ell}$")
    plt.yscale('log')
    
    plt.legend()
    plt.grid()
figps1.set_size_inches(10, 10)

# Normalizing Angular Power Spectra

In [ ]:
print(np.sum(skymaps[0,:], axis=0))

In [ ]:
# normalizing each frequency skymap by total strain of each frequency in attempt to normalize the sph harm coefs.
# Getting total h^2 for each freq
h2_tot_20freqs = np.zeros((20, 1))

for i in range(20): 
    h2_tot_20freqs[i] = np.sum(skymaps[i,:], axis=0)


In [ ]:
print(map_20freqs[0]/h2_tot_20freqs[0])

In [ ]:
# Normalizing
map_20freqs_norm = np.zeros((20, 12288))

for i in range(20):
    map_20freqs_norm[i] = map_20freqs[i]/h2_tot_20freqs[i]


In [ ]:
#new coef calc

c_l_lmax8_20freqs_norm = np.zeros((20, 9))
e_l_lmax8_20freqs_norm = np.zeros((20, 9))


for i in range(20): 
    c_l_lmax8_20freqs_norm[i] = hp.anafast(map_20freqs_norm[i], lmax=8)
    e_l_lmax8_20freqs_norm[i] = np.arange(len(c_l_lmax8_20freqs_norm[i]))

In [ ]:
figps1 = plt.figure()
axps01 = figps1.add_subplot(111)

for i in range(20):
    plt.plot(e_l_lmax8_20freqs_norm[i], e_l_lmax8_20freqs_norm[i] * 
             (e_l_lmax8_20freqs_norm[i] + 1) * c_l_lmax8_20freqs_norm[i], label='freq='+str(i))
    plt.xlabel("$\ell$")
    plt.ylabel("$\ell(\ell+1)C_{\ell}$")
    plt.yscale('log')
    
    plt.legend()
    plt.grid()
figps1.set_size_inches(10, 10)


Plot Just Cl

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for ff in range(F):
    if ff in (0, 10, 20, 30, 39):
        label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
    else: label = None
    ax.plot(l_vals[1:], Cl_lmax8[ff,1:], c=colors[ff], alpha=0.5, label=label,
            marker='o')
ax.set_xlabel('$l$')
ax.set_ylabel('$C_l/C_0$')
ax.set_yscale('log')
ax.set_title('Normalized Spherical Harmonic Coefficients')
ax.legend()
# fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

# Characteristic Strain Plot

making char strain plot: $h_c = (\Sigma h_i^2 \frac{f}{\Delta f})$

In [ ]:
print(np.diff(F_bins).shape)


In [ ]:
char_strain = np.zeros((360))
straintot = np.sum(skymaps[:,:])
print(straintot)

f_bins_plot = F_bins[:-1]
f_bins_size = np.diff(F_bins)
for i in range(360):
    char_strain[i] = (np.sum(skymaps[i,:]) * (f_bins_plot[i]/f_bins_size[i]))**(1/2)

    

print(f_bins_plot.shape)

In [ ]:
figcs = plt.figure()
axcs = figcs.add_subplot(111)

plt.plot(f_bins_plot, char_strain)
# plt.xlabel("$\ell$")
# plt.ylabel("$\ell(\ell+1)C_{\ell}$")
plt.xlim((0,4e-7))
plt.yscale('log')

#plt.legend()
plt.grid()

figps1.set_size_inches(10, 10)

In [ ]:
print(F_bins.shape)